In [1]:
!pip install langdetect
!pip install transformers
!pip install konlpy
! pip install typing
! pip install textrankr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=09e8f8887b13b9e01b60c06f187cd71b4973af4174ec6e062228a4f6c6a50fd2
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (

In [2]:
import pandas as pd
from langdetect import detect

import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
from typing import List
from textrankr import TextRank

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
park = pd.read_csv('/content/drive/MyDrive/BOAZ/mini_project/공원_크롤링.csv')
shopping = pd.read_csv('/content/drive/MyDrive/BOAZ/mini_project/쇼핑몰_크롤링.csv')
market = pd.read_csv('/content/drive/MyDrive/BOAZ/mini_project/시장_크롤링.csv')
landmark = pd.read_csv('/content/drive/MyDrive/BOAZ/mini_project/crawling_landmark_fin.csv')
museum = pd.read_csv('/content/drive/MyDrive/BOAZ/mini_project/crawling_museum.csv')

In [5]:
park

,places,reviews
0,남산 공원,친구들과함게 이른 저녁에 남산타워를 올라 가서 해질녘에 집으로.. 낮도 이쁘지만 해...
1,남산 공원,서울역에서 롯데백화점 뒤쪽 공중정원을 통해 힐튼호텔쪽으로 올라가시면 서울역과 숭례문...
2,남산 공원,주제는 먹방투어 ! 남산 왕돈가스가 유명하다하여 차량네비게이션에 등록한 곳이 아닌 ...
3,남산 공원,뭐 한번 가볼만은 한데 특별하진 않더군요. 케이블카 타느라 많이 기다렸어요. 서울 ...
4,남산 공원,남산 이야기\n\n서울의 지리학적 중심이라고 하면 남산 지역이 아닐까 한다. 남산에...
...,...,...
1023,현대원서공원,I was walking down the street and I saw some s...
1024,현대원서공원,昌徳宮の西側にある公園。現代（ヒュンダイ）社が作ったもののようです。\n階段を上っていくとき...
1025,영등포근린공원,What I find really funny is all these exercise...
1026,영등포근린공원,昼間はおじいさんたちがお友達と将棋をしたり、スポーツしたり子供連れのママさんたちで賑わってい...


In [6]:
shopping

,places,reviews
0,롯데월드타워&몰,일몰 시점에 가면 낮 시간과 일몰의 모습 야경 모두를 볼 수 있습니다 한강과 어울려...
1,롯데월드타워&몰,"우리집은 강동구에 있지만은 조금만 움직이면 롯데타워가\n보인다, 보이는게 문제가 아..."
2,롯데월드타워&몰,입장료가 좀 비싼것 같다.하지만 서울시내 가장 높은 곳에서 내려다 볼 수 있어서 가...
3,롯데월드타워&몰,롯데월드타워. 우리나라에서 가장 높은 전망대입니다. 좋긴 한데 입장료가 좀 비싸고 ...
4,롯데월드타워&몰,이제는 더이상 서울타워도 63빌딩도 롯데타워에 비하면 최고의 높은 전망을 보여주지 ...
...,...,...
852,동대문종합시장,동대문 역에서 굉장히 가깝습니다\n\n원단부터 다양한 의류 부자재들과 함께 항상 사...
853,동대문종합시장,"한 주문이 있을 경우 shopperholic 거대한 쇼핑몰, 원하는 모든 것이 충족..."
854,인사동 마루,여러 art 관련 시설이 한건물에 모여져 있다.\n\n건물 자체도 신관 구관 나뉘어...
855,인사동 마루,우리는이 쇼핑 센터에 매우 실망했습니다. 많은 상점들이 열려 있었고 빈 곳이 많았습...


In [7]:
market

,places,reviews
0,광장시장,Netflex에도 칼국수집 사장님이 나와서 그런지\n핫플레이스가 된듯.. 고객들 대...
1,광장시장,한국 여행의 진정한 맛을 느끼기 위해선 전통 시장에서의 맛 체험이 필수이다. 특히 ...
2,광장시장,동대문 인근의 시중심가의 시장으로 1905년 설립된 국내 최초의 사설 상설시장이다....
3,광장시장,"걷기힘듬\n사람많음\n먹거리 많음 순희네빈대떡, 마약김밥, 떡볶이..등등\n뭐 살 ..."
4,광장시장,워낙 사람이붐비고 유명한곳이라가봤는데\n육회덮밥만먹고왔어요\n생각보다 번잡스럽고 한...
...,...,...
483,구로시장,"전통시장에서 최고의 떡볶이를 맛보시고 싶다면, 구로시장을 추천합니다. 정말 많은 떡..."
484,영동전통시장,골목골목 식자재와 장터 분위기의 맛집들이 숨어있는 강남 한복판 시장이다\n근처를 지...
485,인현시장,인현시장에 있는 통나무집 추천합니다\n1979년도부터 현재까지 40년된 음식점.\n...
486,새마을 전통시장,C ( 20.09.18 )\nThis traditional market was lo...


In [8]:
landmark

,Place,Review
0,명동 쇼핑 거리,코로나19 여파는 매년 늦가을 청계천에서 열리는 아름다운 빛초롱축제도 피해갈 수 없...
1,명동 쇼핑 거리,명동 거리는 늘 사람이 많았는데 코로나 때문인지 ... 사람도 없고 여유롭게 잘 ...
2,명동 쇼핑 거리,대한민국에서 만날 수 있는 모든 형태의 쇼핑을 할 수 있는 곳. 내국인부터 외국인까...
3,명동 쇼핑 거리,Very nice place!! Korea’s famous sandwich fran...
4,명동 쇼핑 거리,관광객들이 명동에 더 많다. 여기에 화장품 샵들이 점차 관광객을 대상으로 영업하더니...
...,...,...
1444,국회 의사당,"아름다운 건물. 한강 바로 옆. 좋은 위치 그리고, 여의도 에서. 많은 음식점들이"
1445,국회 의사당,우리는 한 시간에 그냥 꽃 축제. 벚꽃을 사랑합니다! 아주 아주 깨끗하고 잘 정돈 ...
1446,국회 의사당,이 경우 특히 방문은 한국 정치 에 관심. 운이 좋다면 어떤 설명자 건물에서 볼 수...
1447,국회 의사당,가지 않을 수 있지만 건물을 멋진 사진 및 한국 전통 건물 옆 건물. 전통적인 건물...


In [9]:
landmark.rename(columns={'Place': 'places', 'Review': 'reviews'}, inplace=True)

In [10]:
museum

,Unnamed: 0,Place,Review
0,1449,한국 전쟁 기념관,"여름 서울여행지로 안성맞춤이다.아이들에겐 전쟁역사 공부도 하고, 사진 찍기도 좋다...."
1,1450,한국 전쟁 기념관,대한민국 전쟁의 역사가 기록된 기념관입니다. 건물 자체도 상당히 커서 압도적이고요....
2,1451,한국 전쟁 기념관,아시아에서 가장 큰 전쟁 박물관입니다. 지하철이 옆에 있어 접근하기 좋습니다. 또한...
3,1452,한국 전쟁 기념관,한국전쟁과 관련해서 여러가지 전시물이 있으며 비행기 탱크 등 전쟁에 쓰인 전시...
4,1453,한국 전쟁 기념관,한국사에서 빼놓을 수 없는 것이 전쟁사라고 하죠.. 한 번 쯤은 가보면 굉장히 유익...
...,...,...,...
1129,2578,덕수궁 미술관,"믿을 수 없을만큼 무례하고, 불친절한 직원, 그리고 강요 경비가 입구. 사람들이 그..."
1130,2579,덕수궁 미술관,"도시의 좋은 위치에 있다. 2, 000 KRW 입장료. 좋은 예술 작품. 사원 입구..."
1131,2580,덕수궁 미술관,한국과 해외의 대규모 국제 갤러리에서 모두 사용할 수 있습니다. 무료 사물함 에 환...
1132,2581,덕수궁 미술관,국립 현대 미술관을 방문하는 (MMCA) 의 지사 아리마를 궁 덕수궁 미술관 에 불...


In [11]:
museum.drop(['Unnamed: 0'], axis=1, inplace=True)
museum.rename(columns={'Place': 'places', 'Review': 'reviews'}, inplace=True)

In [12]:
import re

In [13]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'unknown'

def clean_text(text):
  text = re.sub(r'[^\w\s가-힣0-9\s\']', ' ', text)  # 특수문자, 영어 문자, 공백 제거
  text = re.sub(r'[\w\.-]+@[\w\.-]+', ' ', text)  # E-mail 제거
  text = re.sub(r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', ' ', text)  # URL 제거
  text = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]+', ' ', text)  # 한글 자음, 모음 제거
  text = re.sub(r'<[^>]*>', ' ', text)  # HTML 태그 제거
  text = re.sub(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', ' ', text)  # 특수기호 제거
  text = re.sub(r'[a-zA-Z]', ' ', text)  # 영어 문자 제거
  text = re.sub(r'\s+', ' ', text)  # 연속된 공백을 하나의 공백으로 변환
  text = re.sub(r'\n', '.', text)  # 줄바꿈을 마침표로 변환
  return text

In [14]:
park['reviews'] = park['reviews'].apply(clean_text)
park['language'] = park['reviews'].apply(detect_language)
park = park[park['language'] == 'ko']
park.drop(['language'], axis=1, inplace=True)
park

<ipython-input-14-016109a42e09>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  park.drop(['language'], axis=1, inplace=True)


,places,reviews
0,남산 공원,친구들과함게 이른 저녁에 남산타워를 올라 가서 해질녘에 집으로 낮도 이쁘지만 해질녘...
1,남산 공원,서울역에서 롯데백화점 뒤쪽 공중정원을 통해 힐튼호텔쪽으로 올라가시면 서울역과 숭례문...
2,남산 공원,주제는 먹방투어 남산 왕돈가스가 유명하다하여 차량네비게이션에 등록한 곳이 아닌 바로...
3,남산 공원,뭐 한번 가볼만은 한데 특별하진 않더군요 케이블카 타느라 많이 기다렸어요 서울 살지...
4,남산 공원,남산 이야기 서울의 지리학적 중심이라고 하면 남산 지역이 아닐까 한다 남산에 올라갈...
...,...,...
998,사육신공원,먹거리 볼거리 많은 노량진에 위치한 공원 점심식사후 소화를 위해 찾게되는 산책로 복...
999,사육신공원,넘나 좋아요 회사 끝나고 돗자리 들고 가끔 가는데 강바람 시원하고 시끄럽지 않아요 ...
1000,사육신공원,비용 무료 시설 화장실 쉼터 공부방 내용 사육신묘와 쉼터등이 있으면 전망이 좋은 편...
1001,사육신공원,특별히 볼 만한건 없으나 집 근처에 있어서 산책하러 간 곳 이에요 여의도 세계불꽃축...


In [15]:
shopping['reviews'] = shopping['reviews'].apply(clean_text)
shopping['language'] = shopping['reviews'].apply(detect_language)
shopping = shopping[shopping['language'] == 'ko']
shopping.drop(['language'], axis=1, inplace=True)
shopping

<ipython-input-15-59427793ea77>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shopping.drop(['language'], axis=1, inplace=True)


,places,reviews
0,롯데월드타워&몰,일몰 시점에 가면 낮 시간과 일몰의 모습 야경 모두를 볼 수 있습니다 한강과 어울려...
1,롯데월드타워&몰,우리집은 강동구에 있지만은 조금만 움직이면 롯데타워가 보인다 보이는게 문제가 아니라...
2,롯데월드타워&몰,입장료가 좀 비싼것 같다 하지만 서울시내 가장 높은 곳에서 내려다 볼 수 있어서 가...
3,롯데월드타워&몰,롯데월드타워 우리나라에서 가장 높은 전망대입니다 좋긴 한데 입장료가 좀 비싸고 너무...
4,롯데월드타워&몰,이제는 더이상 서울타워도 63빌딩도 롯데타워에 비하면 최고의 높은 전망을 보여주지 ...
...,...,...
852,동대문종합시장,동대문 역에서 굉장히 가깝습니다 원단부터 다양한 의류 부자재들과 함께 항상 사람들이...
853,동대문종합시장,한 주문이 있을 경우 거대한 쇼핑몰 원하는 모든 것이 충족될 수 있다
854,인사동 마루,여러 관련 시설이 한건물에 모여져 있다 건물 자체도 신관 구관 나뉘어져 있는데 무척...
855,인사동 마루,우리는이 쇼핑 센터에 매우 실망했습니다 많은 상점들이 열려 있었고 빈 곳이 많았습니...


In [16]:
market['reviews'] = market['reviews'].apply(clean_text)
market['language'] = market['reviews'].apply(detect_language)
market = market[market['language'] == 'ko']
market.drop(['language'], axis=1, inplace=True)
market

<ipython-input-16-b3a9057c3026>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  market.drop(['language'], axis=1, inplace=True)


,places,reviews
0,광장시장,에도 칼국수집 사장님이 나와서 그런지 핫플레이스가 된듯 고객들 대부분이 외국인들에...
1,광장시장,한국 여행의 진정한 맛을 느끼기 위해선 전통 시장에서의 맛 체험이 필수이다 특히 광...
2,광장시장,동대문 인근의 시중심가의 시장으로 1905년 설립된 국내 최초의 사설 상설시장이다 ...
3,광장시장,걷기힘듬 사람많음 먹거리 많음 순희네빈대떡 마약김밥 떡볶이 등등 뭐 살 것이 있다기...
4,광장시장,워낙 사람이붐비고 유명한곳이라가봤는데 육회덮밥만먹고왔어요 생각보다 번잡스럽고 한복이...
...,...,...
482,구로시장,시장 중앙에 떡볶이집이 몰려있는데 오뎅도 너무 맛있고 떡볶이도 저렴하면서 양도 푸짐...
483,구로시장,전통시장에서 최고의 떡볶이를 맛보시고 싶다면 구로시장을 추천합니다 정말 많은 떡볶이...
484,영동전통시장,골목골목 식자재와 장터 분위기의 맛집들이 숨어있는 강남 한복판 시장이다 근처를 지날...
485,인현시장,인현시장에 있는 통나무집 추천합니다 1979년도부터 현재까지 40년된 음식점 계절별...


In [17]:
landmark['reviews'] = landmark['reviews'].apply(clean_text)
landmark['language'] = landmark['reviews'].apply(detect_language)
landmark = landmark[landmark['language'] == 'ko']
landmark.drop(['language'], axis=1, inplace=True)
landmark

<ipython-input-17-cab493e666bb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  landmark.drop(['language'], axis=1, inplace=True)


,places,reviews
0,명동 쇼핑 거리,코로나19 여파는 매년 늦가을 청계천에서 열리는 아름다운 빛초롱축제도 피해갈 수 없...
1,명동 쇼핑 거리,명동 거리는 늘 사람이 많았는데 코로나 때문인지 사람도 없고 여유롭게 잘 쇼핑하다 ...
2,명동 쇼핑 거리,대한민국에서 만날 수 있는 모든 형태의 쇼핑을 할 수 있는 곳 내국인부터 외국인까지...
3,명동 쇼핑 거리,명동에서도 맛볼수 있는 한국의 유명한 샌드위치 프랜차이즈 포슬포슬한 에그 스크램블...
4,명동 쇼핑 거리,관광객들이 명동에 더 많다 여기에 화장품 샵들이 점차 관광객을 대상으로 영업하더니 ...
...,...,...
1444,국회 의사당,아름다운 건물 한강 바로 옆 좋은 위치 그리고 여의도 에서 많은 음식점들이
1445,국회 의사당,우리는 한 시간에 그냥 꽃 축제 벚꽃을 사랑합니다 아주 아주 깨끗하고 잘 정돈 된 ...
1446,국회 의사당,이 경우 특히 방문은 한국 정치 에 관심 운이 좋다면 어떤 설명자 건물에서 볼 수 있다
1447,국회 의사당,가지 않을 수 있지만 건물을 멋진 사진 및 한국 전통 건물 옆 건물 전통적인 건물이...


In [18]:
park = park.groupby(['places'])['reviews'].apply(' '.join).reset_index()
shopping = shopping.groupby(['places'])['reviews'].apply(' '.join).reset_index()
market = market.groupby(['places'])['reviews'].apply(' '.join).reset_index()
landmark = landmark.groupby(['places'])['reviews'].apply(' '.join).reset_index()
museum = museum.groupby(['places'])['reviews'].apply(' '.join).reset_index()

# BERT

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [20]:
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

tokenizer = BertTokenizer.from_pretrained("monologg/kobert")
model = BertModel.from_pretrained("monologg/kobert")
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(8002, 768, padding_idx=1)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
   

In [24]:
def embed_text(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
    inputs = inputs.to(device)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().detach().cpu().numpy()

In [25]:
def get_top_similar_places(selected_place, similarity_df, top_k=3):
    similar_places = similarity_df[selected_place].sort_values(ascending=False)[1:top_k+1]
    return similar_places

In [26]:
embeddings_list = []
places_list = []

select = input('카테고리 입력(공원, 쇼핑몰, 시장, 랜드마크, 박물관): ')

if select == "공원":
  category = park
if select == "쇼핑몰":
  category = shopping
if select == "시장":
  category = market
if select == "랜드마크":
  category = landmark
if select == "박물관":
  category = museum

for index, row in category.iterrows():
    place = row['places']
    reviews = row['reviews']
    embeddings = embed_text(reviews)
    embeddings_list.append(embeddings)
    places_list.append(place)

embeddings_tensor = torch.tensor(embeddings_list, device=device)

similarity_matrix = cosine_similarity(embeddings_tensor.cpu().numpy())

similarity_df = pd.DataFrame(similarity_matrix, index=places_list, columns=places_list)
print("Cosine Similarity Matrix:")
print(similarity_df)

카테고리 입력(공원, 쇼핑몰, 시장, 랜드마크, 박물관): 랜드마크
Cosine Similarity Matrix:
              N 서울 타워     고려대학교      광장시장       광화문    광화문 광장    국회 의사당  \
N 서울 타워      1.000000  0.980237  0.978987  0.969036  0.973159  0.969853   
고려대학교        0.980237  1.000000  0.980860  0.977979  0.976477  0.980021   
광장시장         0.978987  0.980860  1.000000  0.963287  0.968244  0.965281   
광화문          0.969036  0.977979  0.963287  1.000000  0.982612  0.980170   
광화문 광장       0.973159  0.976477  0.968244  0.982612  1.000000  0.978154   
국회 의사당       0.969853  0.980021  0.965281  0.980170  0.978154  1.000000   
덕수궁          0.956749  0.970658  0.978095  0.960215  0.962604  0.955019   
동대문 (흥인지문)   0.959449  0.971732  0.977399  0.966006  0.971545  0.961858   
동대문 디자인 플라자  0.970436  0.982720  0.963920  0.979478  0.982721  0.974774   
롯데월드타워&몰     0.978149  0.982344  0.969451  0.984754  0.980727  0.984924   
명동 쇼핑 거리     0.958224  0.960281  0.939297  0.958235  0.965895  0.954021   
봉은사          0.971846  0.982786  0.9

In [27]:
selected_place = input()   # 사용자가 선택한 장소

top_similar_places = get_top_similar_places(selected_place, similarity_df)
print(f"{selected_place}와 유사도가 가장 높은 장소 3개:")
print(top_similar_places)

명동 쇼핑 거리
명동 쇼핑 거리와 유사도가 가장 높은 장소 3개:
동대문 디자인 플라자    0.970207
서울 대학로         0.965961
광화문 광장         0.965895
Name: 명동 쇼핑 거리, dtype: float32


# Doc2vec

In [28]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [29]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [30]:
select = input('카테고리 입력(공원, 쇼핑몰, 시장, 랜드마크, 박물관): ')

if select == "공원":
  df = park
if select == "쇼핑몰":
  df = shopping
if select == "시장":
  df = market
if select == "랜드마크":
  df = landmark
if select == "박물관":
  df = museum

카테고리 입력(공원, 쇼핑몰, 시장, 랜드마크, 박물관): 쇼핑몰


In [31]:
# 텍스트 전처리 (토큰화 및 정규화)
documents = [
    TaggedDocument(words=word_tokenize(review), tags=[place])
    for place, review in zip(df['places'], df['reviews'])
]
# doc2vec 모델 학습
model = Doc2Vec(vector_size=20, window=2, min_count=1, workers=4, epochs=100)
model.build_vocab(documents)
model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)

In [32]:
# 사용자 입력 처리
user_input = input("여행지 입력: ")
user_keywords = word_tokenize(user_input.lower())

# 사용자 입력을 벡터로 변환
user_embedding = model.infer_vector(user_keywords)

여행지 입력: 인사동 마루


In [33]:
similarities = []
for _, row in df.iterrows():
    place = row['places']
    review = row['reviews']

    place_embedding = model.dv[place]
    review_tokens = word_tokenize(review)  # 리뷰 문장을 토큰화
    review_embedding = model.infer_vector(review_tokens)

    similarity = cosine_similarity([user_embedding], [review_embedding])[0][0]
    similarities.append((place, similarity))

In [34]:
# 유사도가 높은 순서대로 정렬
similarities = sorted(similarities, key=lambda x: x[1], reverse=True)

In [35]:
print(f"'{user_input}'와(과) 유사한 여행지 추천:")
for place, similarity in similarities[:3]:  # 상위 5개만 출력
    print(f"{place}: 유사도 = {similarity:.4f}")

'인사동 마루'와(과) 유사한 여행지 추천:
쌈지길: 유사도 = 0.8505
회현 지하 상가: 유사도 = 0.7965
퀸마마마켓: 유사도 = 0.7466
